# Logistic Regression
Let's try to implement a logistic regression fit using neural network approach.

In [1]:
# setup
import numpy as np

In [2]:
# sigmoid
def sigmoid(x):
    return(1 / (1 + np.exp(-x)))

# test
sigmoid(np.array([0,1,2,3]))

array([ 0.5       ,  0.73105858,  0.88079708,  0.95257413])

In [4]:
# let's generate some simulated data

# input
n_trials = 1000
x = np.random.uniform(size=n_trials, low=-5, high = 5).reshape((1,n_trials))

# train output
y = np.random.binomial(1,sigmoid(0.8*x-0.5))

print(np.shape(x))
print(x)
print(sigmoid(0.8*x+0.5))
print(y)

(1, 1000)
[[-3.35110462  0.32875429  3.20898186 -3.32745387  4.60875456  4.45980096
  -4.36653201 -0.42708365  1.18815978 -2.8090016  -3.8011879  -3.6050737
   4.48361921  4.27598109  2.46137052 -0.57949119  3.26847868  3.34564746
  -4.74789624  0.29509296  2.39023492 -0.94869024  0.72892305 -3.4539802
   3.57526958 -0.67703526  4.31014729  2.01007114 -4.47694395 -4.22081633
  -4.9022604   2.30692304  0.48858751  1.49478139  3.30840881  3.17432421
   2.35480053  2.62957765 -2.53967122  0.23462921  4.98205538 -4.59050008
   2.92291732 -0.7511206  -0.92701301  2.50773881  3.66758004  4.65554386
  -3.60633654 -3.10356913  3.08746099  1.3699179  -0.01305225  4.58538247
   3.01049565 -4.61362943  3.88102579 -4.82081532 -2.64508582  1.23144114
   3.61146711  4.22072626 -3.92875277 -3.80099762 -0.63281813 -1.11890149
   3.55094588 -4.59553788 -0.72469769  3.42767971  4.25097399  3.63781914
   3.20059186 -1.89027191  1.52368784 -1.52579858 -3.9888788  -4.44161353
  -2.74747085  2.4491113  -1.2

In [ ]:
# z_i = w*x_i + b
# z = w*x + b
# dz/dw = x
# dz/db = 1
# a = yhat = sigmoid(z)
# da/dz = a*(1-a)

# likelihood - want to maximize that
# l = a^y * (1-a)^(1-y)

# loss function - want to minimize hence "minus"
# -log(l) = -( y*log(a) + (1-y)*log(1-a) )

# cost function
# L = -1/m * sum(log(l))

# gradient descent in the notation of NN back propagation
# dz := dL/dz = dL/da*da/dz = -1/m * sum( (y/a + (y-1)/(1-a))*a*(1-a) ) = -1/m * sum(y-a)
# dw := dL/dw = dL/dz * dz/dw := dz * x
# db := dL/db = dL/dz * dz/db := dz
# step
# w -> w - alpha * x * dz
# b -> b - alpha * dz

In [5]:
# starting point
alpha=1
m = np.shape(x)[1]
w = np.ones((1,1))
#w = 1
b = 2
print(m)
print('learning rate:',alpha)
np.shape(w)

1000
learning rate: 1


(1, 1)

In [6]:
dz=1
i=0
w = np.random.uniform(high=10,low=-10,size=1).reshape((1,1))
b = np.random.uniform(high=10,low=-10,size=1)
print(m,w,b)
while(abs(dz)>0.0000001):
#for i in range(1000):
    i += 1
    
    # forward step
    z = np.dot(w.T,x) + b
    A = sigmoid(z)
    cost = -1./m * (np.dot(y,np.log(A).T) + np.dot((1-y),np.log(1-A).T))
    
    # back propagation
    dz = 1./m * np.sum(A-y)
    dw = 1./m * np.dot(x,(A-y).T) 
    db = dz
    
    w = w - alpha * dw
    b = b - alpha * db

    if i==0:
        print('z:',z)
        print('dz:',dz)
        print('dw:',dw)
    
    if i>100000:
        break
    
print(i, dz)
w,b

1000 [[-6.35794674]] [ 6.69886699]
141 9.61773037647e-08


/home/kukarzev/venv_flask/lib/python3.5/site-packages/ipykernel/__main__.py:13: RuntimeWarning: divide by zero encountered in log


(array([[ 0.73347818]]), array([-0.48646616]))

In [7]:
import pandas as pd
df = pd.DataFrame({'x':x[0], 'y':y[0]})
df.head()

,x,y
0,-3.351105,0
1,0.328754,1
2,3.208982,1
3,-3.327454,1
4,4.608755,1


In [9]:
# compare to sklearn
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression

yy, XX = dmatrices('y ~ x', df, return_type = 'dataframe')

model = LogisticRegression(max_iter=100000,tol=0.0000001)
model.fit(XX,yy)
model.coef_

/home/kukarzev/venv_flask/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[-0.24179933,  0.73189205]])

In [10]:
# check with statsmodels
import statsmodels.discrete.discrete_model as sm
logit = sm.Logit(yy, XX)
logit.fit().params

Optimization terminated successfully.
         Current function value: 0.395426
         Iterations 7


Intercept   -0.486467
x            0.733478
dtype: float64